In [107]:
import requests
import time
import pandas as pd
import bs4
import matplotlib.pyplot as plt
import seaborn as sns
from deepdiff import DeepDiff

### get_page
params: url str<br>
returns: soup obj

In [108]:
def get_page(url):
    request = requests.get(url)
    request.encoding = None
    numsoup = bs4.BeautifulSoup(request.text, "html.parser")
    time.sleep(2)
    return numsoup

### text_list
params: soup obj<br>
returns: list str

In [109]:
def text_list(x):
    x = x.text.split("\n")
    new = []
    for val in x:
        if val != "" and val != "\xa0\xa0":
            new.append(val)
    return new

### read_file
params: file_name of file in the same directory str<br>
scrape_dict dictionary str<br>
returns: dictionary from file to compare to scraped dictionary str

In [110]:
def read_file(file_name, scrape_dict):
    
    f = open(file_name, "r")

    file_dict = {}
    for key in scrape_dict:
        file_dict[key] = ""

    file = []
    index = []
    line = f.readline()
    file.append(line.strip("\n"))
    x = []
    x.append(file.index(line.strip("\n")))

    while line:
        line = f.readline() 
        line = line.strip("\n")
        file.append(line)
        if line in file_dict:
            x.append(file.index(line))

    for i in range(len(x)):
        if i+1 < len(x):
            file_dict[file[x[i]]] = file[x[i]+1:x[i+1]]
        else:
            file_dict[file[x[i]]] = file[x[i]+1:-1]

    f.close()
    
    return file_dict

### get_differences
params: text dictionary str<br>
scrape_dict dictionary str<br>
name of website str<br>
returns: na, prints differences<br>
calls write_to_file if there are differences to update txt file

In [111]:
def get_differences(text_dict, scraped_dict, name, file_name):
    diff = DeepDiff(text_dict, scraped_dict)
    print(name+":")
    if diff == {}:
        print("Nothing new to report!")
    else:
        print(diff)
        write_to_file(file_name, scrape_dict)

### write_to_file
params: text dictionary str<br>
file name str<br>
returns: na

In [112]:
def write_to_file(file_name, scrape_dict):
    f = open(file_name, "w")
    for key in scrape_dict:
        f.write(key+"\n")
        for val in scrape_dict[key]:
            f.write(val+"\n")
    f.close()

## ICLR

In [113]:
def ICLR():
    
    iclr_url = "https://iclr.cc/Conferences/2023"
    iclr_titles = {"Registration":"", "Announcements":"","Important Dates":""}
    soup = get_page(iclr_url)

    parent = soup.find_all(class_="col-sm-6")
    registration = parent[1].find_all("div")
    announcements = parent[2]
    registration[2]
    parent_dates = soup.find_all(class_="col-xs-12")
    dates = parent_dates[1].find_all("span")

    registration = text_list(registration[0])
    iclr_titles["Registration"] = registration[1:]
    announcements = text_list(announcements)
    iclr_titles["Announcements"] = announcements[1:]

    new_dates = []
    for val in dates:
        if val.text != "" and val.text != "*":
            new_dates.append(val.text)
    iclr_titles["Important Dates"] = new_dates

    text_dict = read_file("iclr.txt", iclr_titles)

    get_differences(text_dict, iclr_titles, "ICLR", "iclr.txt")

In [114]:
ICLR()

ICLR:
Nothing new to report!
